In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/dataset1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers liqfit

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from sklearn.model_selection import train_test_split
# import pandas as pd

# # Step 1: Load your CSV dataset
# data = pd.read_csv('dataset101.csv',encoding='cp1252')

# # Step 2: Split the dataset into training and validation sets
# train_texts, val_texts, train_labels, val_labels = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# # Step 3: Load pre-trained model and tokenizer
# model_name = "knowledgator/comprehend_it-base"  # Example model, you can replace it with your desired model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=10, ignore_mismatched_sizes=True)  # Add ignore_mismatched_sizes=True

# # Step 4: Tokenization
# train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
# val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# # Step 5: Fine-tuning
# training_args = TrainingArguments(
#     output_dir='./results',  # Specify output directory
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,  # You can adjust the number of epochs
#     logging_dir='./logs',
#     logging_steps=100,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_encodings,
#     eval_dataset=val_encodings,
# )

# trainer.train()

# # Step 6: Evaluation
# result = trainer.evaluate()
# print(result)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from liqfit.datasets import NLIDataset
from liqfit.collators import NLICollator
from datasets import load_dataset

In [ ]:
model_path = 'knowledgator/comprehend_it-base'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained('knowledgator/comprehend_it-base')

In [ ]:
data_collator = NLICollator(tokenizer, max_length=128, padding=True, truncation=True)

#change the params accdly
training_args = TrainingArguments(
    output_dir='comprehendo',
    learning_rate=3e-5,
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    num_train_epochs=9,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_steps = 5000,
    save_total_limit=3,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=nli_train_dataset,
    eval_dataset=nli_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

## test the fine-tuned model in a few-shot setting with the zero-shot-classification pipeline of the transformers librar

In [ ]:
# from transformers import pipeline
# from sklearn.metrics import classification_report
# from tqdm import tqdm
# import torch

# device = torch.device('cuda:0')

# classifier = pipeline("zero-shot-classification",
#                       model=model,tokenizer=tokenizer, device=device)

# from tqdm import tqdm

# label2idx = {label: id for id, label in enumerate(classes)}
# test_dataset = dataset #give path here
# preds = []
# template = 'This example is {}.'
# new_classes =  [template.format(c) for c in classes]
# label2idx = {label: id for id, label in enumerate(new_classes)}

# for example in tqdm(test_dataset):
#     if not example['text']:
#         preds.append(idx)
#         continue
#     pred = classifier(''+example['text'],new_classes, hypothesis_template = template)['labels'][0]
#     idx = label2idx[pred]
#     preds.append(idx)

# print(classification_report(test_dataset['label'][:len(preds)], preds, target_names=classes, digits=4))

In [ ]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("dataset1.csv",encoding='cp1252')
classes = df['label'].unique().tolist()
dataset_dict = {'text': df['text'], 'label': df['label']}
dataset = Dataset.from_dict(dataset_dict)

classes = df['label'].unique().tolist()
nli_train_dataset = NLIDataset.load_dataset(dataset, classes=classes)
nli_test_dataset = NLIDataset.load_dataset(dataset, classes=classes)